<a href="https://colab.research.google.com/github/siahw/notes/blob/main/%E2%80%9C2_Named_Entity_Recognition_Livecoding_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition
- For a given word and its context window, estimate whether the given word is 

# 1. Download dataset
- CoNLL2003 

In [ ]:
!wget https://data.deepai.org/conll2003.zip # Download dataset
!unzip conll2003.zip # Unzip dataset zip

## 2. Preprocess Dataset

In [ ]:
with open("train.txt") as f:
  string = ''.join(f.readlines())
dataset = string.split('\n')

dataset[50:80]  #I-PER means continuous for the previous class is related to this current word

["'s POS B-NP O",
 'representative NN I-NP O',
 'to TO B-PP O',
 'the DT B-NP O',
 'European NNP I-NP B-ORG',
 'Union NNP I-NP I-ORG',
 "'s POS B-NP O",
 'veterinary JJ I-NP O',
 'committee NN I-NP O',
 'Werner NNP I-NP B-PER',
 'Zwingmann NNP I-NP I-PER',
 'said VBD B-VP O',
 'on IN B-PP O',
 'Wednesday NNP B-NP O',
 'consumers NNS I-NP O',
 'should MD B-VP O',
 'buy VB I-VP O',
 'sheepmeat NN B-NP O',
 'from IN B-PP O',
 'countries NNS B-NP O',
 'other JJ B-ADJP O',
 'than IN B-PP O',
 'Britain NNP B-NP B-LOC',
 'until IN B-SBAR O',
 'the DT B-NP O',
 'scientific JJ I-NP O',
 'advice NN I-NP O',
 'was VBD B-VP O',
 'clearer JJR B-ADJP O',
 '. . O O']

In [ ]:
from itertools import groupby

dataset_in_sentence = [list(group) for k, group in groupby(dataset, lambda x: x == "") if not k]
dataset_in_sentence[:5]

[['-DOCSTART- -X- -X- O'],
 ['EU NNP B-NP B-ORG',
  'rejects VBZ B-VP O',
  'German JJ B-NP B-MISC',
  'call NN I-NP O',
  'to TO B-VP O',
  'boycott VB I-VP O',
  'British JJ B-NP B-MISC',
  'lamb NN I-NP O',
  '. . O O'],
 ['Peter NNP B-NP B-PER', 'Blackburn NNP I-NP I-PER'],
 ['BRUSSELS NNP B-NP B-LOC', '1996-08-22 CD I-NP O'],
 ['The DT B-NP O',
  'European NNP I-NP B-ORG',
  'Commission NNP I-NP I-ORG',
  'said VBD B-VP O',
  'on IN B-PP O',
  'Thursday NNP B-NP O',
  'it PRP B-NP O',
  'disagreed VBD B-VP O',
  'with IN B-PP O',
  'German JJ B-NP B-MISC',
  'advice NN I-NP O',
  'to TO B-PP O',
  'consumers NNS B-NP O',
  'to TO B-VP O',
  'shun VB I-VP O',
  'British JJ B-NP B-MISC',
  'lamb NN I-NP O',
  'until IN B-SBAR O',
  'scientists NNS B-NP O',
  'determine VBP B-VP O',
  'whether IN B-SBAR O',
  'mad JJ B-NP O',
  'cow NN I-NP O',
  'disease NN I-NP O',
  'can MD B-VP O',
  'be VB I-VP O',
  'transmitted VBN I-VP O',
  'to TO B-PP O',
  'sheep NN B-NP O',
  '. . O O']]

### 2.1 Make Data Loader

In [ ]:
window_size = 2 
sentence = dataset_in_sentence[4]
word_idx = 5 #5th list of word
center_word = sentence[2].split(' ')[0] #select the first word

# from word_idx - window_length until word_idx + window_length
def get_words_window(sentence, word_idx, window_size):
  idx_range = range( max(word_idx-window_size, 0), min(word_idx+window_size+1, len(sentence)))
  windowed_words = [sentence[idx].split(' ')[0].lower() for idx in idx_range]
  
  if word_idx < window_size:
    windowed_words = ['PAD'] * (window_size-word_idx) + windowed_words
  if word_idx + window_size >= len(sentence):
    windowed_words += ['PAD'] * (word_idx + window_size -len(sentence)+1)
  
  return windowed_words

get_words_window(sentence, word_idx, window_size)
# center_word

['said', 'on', 'thursday', 'it', 'disagreed']

In [ ]:
import gensim.downloader

wrd2vec = gensim.downloader.load("glove-wiki-gigaword-300")

In [ ]:
word_idx = 0
words_window = get_words_window(sentence, word_idx, window_size)
print(words_window)
for word in words_window:
  if word in wrd2vec:
    print(word , wrd2vec[word][:10])

['PAD', 'PAD', 'the', 'european', 'commission']
the [ 0.04656    0.21318   -0.0074364 -0.45854   -0.035639   0.23643
 -0.28836    0.21521   -0.13486   -1.6413   ]
european [ 0.37636   0.1245    0.13028   0.024309  0.50706   0.18205  -0.44874
  0.35522   0.27065  -2.2199  ]
commission [ 4.7286e-01 -3.9634e-01 -2.6584e-01  1.2371e-02 -2.5906e-01 -2.5823e-01
  1.1492e-01 -1.4363e-01  2.0568e-03 -2.2627e+00]


In [ ]:
# concat vectors
import torch

concat_vector = []
for word in words_window:
  if word in wrd2vec:
    concat_vector.append(torch.Tensor(wrd2vec[word]))
  else:
    zero_vector = torch.zeros(300) #300dimension
    concat_vector.append(zero_vector)

torch.cat(concat_vector).shape

torch.Size([1500])

In [ ]:
concat_tensor = torch.cat(concat_vector)

print(concat_tensor[650] == concat_vector[2][49]) 
'''
[300 300 300 300 300] #5 300vectors
1500 

concat_tensor's 1200th dimension
= 0th dimension of the 4th word because 300*4=12000

concat_tensor's 50th dimension
= 50th dimension of the 0th word becasue 0+50=50

concat_tensor's 350th dimension
= 50th dimension of the 1st word because 300+50=350


'''

tensor(False)


"\n[300 300 300 300 300]\n1500 \n\nconcat_tensor's 1200th dimension\n= 0th dimension of the 4th word\n\nconcat_tensor's 50th dimension\n= 50th dimension of the 0th word\n\nconcat_tensor's 350th dimension\n= 50th dimension of the 1st word\n\n\n"

In [ ]:
import torch.nn as nn #multilayer perception

hidden_size = 128
first_layer = nn.Linear(300*5, 128, bias=False)

In [ ]:
first_layer.weight.shape #128 neurons, each neurons see 1500vectors

torch.Size([128, 1500])

In [ ]:
hidden = first_layer(concat_tensor)
concat_tensor.shape, hidden.shape

(torch.Size([1500]), torch.Size([128]))

In [ ]:
torch.matmul(first_layer.weight, concat_tensor)

tensor([-0.1014, -0.0434,  0.1495,  0.0594, -0.1511, -0.0104, -0.0830, -0.0217,
         0.0269,  0.1687,  0.2679,  0.0423,  0.1710, -0.0205, -0.1302, -0.3823,
        -0.2216,  0.1646,  0.2188,  0.0234, -0.1152,  0.1256,  0.1081,  0.2385,
         0.2288, -0.0017,  0.0467, -0.0243,  0.1805,  0.0397,  0.0331,  0.1226,
         0.1068, -0.1220,  0.2065, -0.1513,  0.2422,  0.1978,  0.0621,  0.0944,
         0.0589,  0.0851,  0.2401,  0.0359, -0.1163,  0.1025, -0.3653,  0.1052,
         0.1608,  0.0969, -0.0260, -0.1708, -0.2358, -0.2242, -0.0224, -0.0895,
         0.0413, -0.2420, -0.0761, -0.0006,  0.1911,  0.0721, -0.0274,  0.0084,
         0.1376, -0.0117, -0.0238, -0.0739,  0.0417, -0.0189,  0.0175,  0.0612,
         0.2436, -0.0954,  0.0930, -0.1291,  0.3059, -0.2236, -0.1336,  0.1935,
         0.2372, -0.0942, -0.0061, -0.2416,  0.1701,  0.1200,  0.2148, -0.0873,
        -0.0901, -0.1089, -0.1481, -0.0412,  0.2116, -0.0769, -0.1613, -0.1882,
         0.1070, -0.1290, -0.0027, -0.04

In [ ]:
first_layer(concat_tensor)

tensor([-0.1014, -0.0434,  0.1495,  0.0594, -0.1511, -0.0104, -0.0830, -0.0217,
         0.0269,  0.1687,  0.2679,  0.0423,  0.1710, -0.0205, -0.1302, -0.3823,
        -0.2216,  0.1646,  0.2188,  0.0234, -0.1152,  0.1256,  0.1081,  0.2385,
         0.2288, -0.0017,  0.0467, -0.0243,  0.1805,  0.0397,  0.0331,  0.1226,
         0.1068, -0.1220,  0.2065, -0.1513,  0.2422,  0.1978,  0.0621,  0.0944,
         0.0589,  0.0851,  0.2401,  0.0359, -0.1163,  0.1025, -0.3653,  0.1052,
         0.1608,  0.0969, -0.0260, -0.1708, -0.2358, -0.2242, -0.0224, -0.0895,
         0.0413, -0.2420, -0.0761, -0.0006,  0.1911,  0.0721, -0.0274,  0.0084,
         0.1376, -0.0117, -0.0238, -0.0739,  0.0417, -0.0189,  0.0175,  0.0612,
         0.2436, -0.0954,  0.0930, -0.1291,  0.3059, -0.2236, -0.1336,  0.1935,
         0.2372, -0.0942, -0.0061, -0.2416,  0.1701,  0.1200,  0.2148, -0.0873,
        -0.0901, -0.1089, -0.1481, -0.0412,  0.2116, -0.0769, -0.1613, -0.1882,
         0.1070, -0.1290, -0.0027, -0.04

In [ ]:
print(first_layer.weight.shape)
first_layer.weight[0]

torch.Size([128, 1500])


tensor([ 0.0095, -0.0012,  0.0064,  ...,  0.0005,  0.0175,  0.0248],
       grad_fn=<SelectBackward0>)

In [ ]:
entire_output = []
for neuron in first_layer.weight:
  entire_output.append((neuron * concat_tensor).sum())
# entire_output
                    

In [ ]:
words_window

['PAD', 'PAD', 'the', 'european', 'commission']

In [ ]:
first_layer.weight[0][600:900] #weight for the 600~900 which is 'the' dimension for 0th word
#run the previous code: words_window
#[0:300]will be PAD
#[300:600]will be PAD
#[600:900]will be the
#[900:1200]will be european

tensor([ 0.0054,  0.0077, -0.0163, -0.0124,  0.0235, -0.0194,  0.0113, -0.0191,
        -0.0092, -0.0189, -0.0028, -0.0012,  0.0167, -0.0037,  0.0132, -0.0157,
        -0.0244, -0.0151, -0.0135, -0.0045,  0.0148, -0.0084,  0.0058, -0.0123,
        -0.0146,  0.0097, -0.0033, -0.0251,  0.0018, -0.0038,  0.0188,  0.0170,
         0.0231,  0.0035,  0.0183, -0.0138,  0.0166,  0.0120, -0.0095, -0.0099,
        -0.0049,  0.0172,  0.0048,  0.0203, -0.0244, -0.0243,  0.0124, -0.0147,
        -0.0240,  0.0228, -0.0096,  0.0235,  0.0008,  0.0127,  0.0087,  0.0065,
         0.0162, -0.0243, -0.0118, -0.0236, -0.0251,  0.0103, -0.0053,  0.0166,
         0.0233,  0.0222,  0.0202,  0.0182,  0.0192,  0.0250, -0.0059,  0.0014,
        -0.0155,  0.0045, -0.0057, -0.0007, -0.0106, -0.0187,  0.0243, -0.0133,
         0.0140, -0.0190,  0.0144,  0.0083, -0.0186,  0.0147,  0.0073,  0.0174,
         0.0132,  0.0005, -0.0211, -0.0121,  0.0253,  0.0171,  0.0217,  0.0058,
         0.0232,  0.0018,  0.0053, -0.02

In [ ]:
hidden = first_layer(concat_tensor)
hidden_after_activation = torch.relu(hidden) #relu rotate and squeeze data
hidden[:20], hidden_after_activation[:20]

(tensor([-0.1014, -0.0434,  0.1495,  0.0594, -0.1511, -0.0104, -0.0830, -0.0217,
          0.0269,  0.1687,  0.2679,  0.0423,  0.1710, -0.0205, -0.1302, -0.3823,
         -0.2216,  0.1646,  0.2188,  0.0234], grad_fn=<SliceBackward0>),
 tensor([0.0000, 0.0000, 0.1495, 0.0594, 0.0000, 0.0000, 0.0000, 0.0000, 0.0269,
         0.1687, 0.2679, 0.0423, 0.1710, 0.0000, 0.0000, 0.0000, 0.0000, 0.1646,
         0.2188, 0.0234], grad_fn=<SliceBackward0>))

In [ ]:
hidden.shape

torch.Size([128])

In [ ]:
second_layer = nn.Linear(in_features=128, out_features=1)


tensor([-0.0112], grad_fn=<AddBackward0>)

In [ ]:
second_output = second_layer(hidden_after_activation)
final_prediction = torch.sigmoid(second_output)

words_window, final_prediction
#run the code
#it means 0.4972 percent it predicts that it is the name entitiy of LOC 

(['PAD', 'PAD', 'the', 'european', 'commission'],
 tensor([0.4972], grad_fn=<SigmoidBackward0>))

In [ ]:
torch.sigmoid(torch.zeros(1))

tensor([0.5000])

In [ ]:
get_words_window(sentence, word_idx, window_size)


In [ ]:
sentence[:5]

['The DT B-NP O',
 'European NNP I-NP B-ORG',
 'Commission NNP I-NP I-ORG',
 'said VBD B-VP O',
 'on IN B-PP O']

## 3. Make a Neural Network Model

## 4. Implement a train loop

## 5. Make Inference